In [15]:
import os
from keras.preprocessing import image
import cv2



In [16]:
from keras.preprocessing import image
import cv2

In [1]:
categories=['with_mask','without_mask']

Download the dataset from : https://www.kaggle.com/datasets/omkargurav/face-mask-dataset


Create the folder name Image datset and inside it there is two folder with_mask and without_mask which contains their respective images


In [ ]:
##initialize the empty list for image and their corresponding label
data=[]


for category in categories:
    ## get the index of the category for eg the index of with_mask is 0 and without_mask is 1
    label=categories.index(category)
    ##label =0
    ## The os.path.join function in Python is used to concatenate multiple path components into a single path. In windows \ is used as path separator.
    path= os.path.join('Image dataset', category)
    ## path = Image dataset\ with_mask
    
    ##Loop through each file in the directory specified by path
    for file in os.listdir(path):
        ##Construct the full path to the current image file
        image_path= os.path.join(path, file)
        ##read the image using cv2
        image= cv2.imread(image_path)
        ##resize the image to 224,224
        image= cv2.resize(image, (224,224))
        ## append the image and  its respective label to the data which is list
        data.append([image,label])
        
        


In [ ]:
## total number of images in the dataset
len(data)

7553

In [ ]:
## shuffle the data( images and their corresponding label) so that the model does not learn the order of the images
import random
random.shuffle(data)



In [ ]:
##seperate the images and their corresponding label into X and y
X=[]
y=[]
for features, label in data:
    X.append(features)
    y.append(label)

In [ ]:
import numpy as np
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(7553, 224, 224, 3)

In [ ]:
X= X/255

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=0) 

In [ ]:
X_train.shape

(6042, 224, 224, 3)

In [ ]:
X_train.shape

(6042, 224, 224, 3)

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg=VGG16()

In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

 block3_pool (MaxPooling2D)  (None, 28, 28, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 28, 28, 512)       1180160   
                                                                 
 block4_conv2 (Conv2D)       (None, 28, 28, 512)       2359808   
                                                                 
 block4_conv3 (Conv2D)       (None, 28, 28, 512)       2359808   
                                                                 
 block4_pool (MaxPooling2D)  (None, 14, 14, 512)       0         
                                                                 
 block5_conv1 (Conv2D)       (None, 14, 14, 512)       2359808   
                                                                 
 block5_conv2 (Conv2D)       (None, 14, 14, 512)       2359808   
                                                                 
 block5_conv3 (Conv2D)       (None, 14, 14, 512)       2359808   
          

In [ ]:
from keras import Sequential

In [ ]:
model=Sequential()

In [ ]:
## add all the layers of vgg except the last layer
## VGG is functional API but we need sequential model so we did not add the last layer
for layer in vgg.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
## we freeze all the layers of vgg so that the weights of the vgg are not updated during training 
## we are doing transfer learning

for layer in model.layers:
    layer.trainable=False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
from keras.layers import Dense

In [ ]:
## add the last layer of the model which is dense layer with 1 output node and sigmoid activation function
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 64

model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test))

Test the model on a single image


In [ ]:
test_img= cv2.imread('mask wala.jpg')
test_img= cv2.resize(test_img, (224,224))

In [ ]:
cv2.imshow('image', test_img)

In [ ]:
test_pred = model.predict(test_img.reshape(1,224,224,3))

1/1 [==============================] - 0s 235ms/step


In [ ]:
print(test_pred)

[[5.971273e-08]]


Acccess the camera and detect the mask


In [ ]:
import cv2
cap= cv2.VideoCapture(0)

In [ ]:
def draw_label(img, text, pos, bg_color):
    textsize = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    end_X = pos[0] + textsize[0][0] + 2
    end_Y = pos[1] - textsize[0][1] - 2
    cv2.rectangle(img, pos, (end_X, end_Y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)


def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1, 224, 224, 3))
    return y_pred[0][0]


while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224, 224))
    y_pred = detect_face_mask(img)
    
    if y_pred == 0:
        draw_label(frame, "Mask Detected", (30, 30), (0, 255, 0))
    else:
        draw_label(frame, "Mask Not Detected", (30, 30), (0, 0, 255))
        
    cv2.imshow('Window', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()
